# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../my_output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed
0,jamestown,-33.21,138.60,13.47,87,89,2.65
1,knysna,-34.04,23.05,19.00,93,100,1.00
2,meulaboh,4.14,96.13,26.89,81,0,3.03
3,jingdezhen,29.29,117.20,12.33,93,77,1.13
4,sioux lookout,50.10,-91.92,-5.00,92,90,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key= g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Add Humidity as weight
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# My ideal vacation spot has temperatures between 15-22(c) or 60-70(f), wind speeds above 10mph, and is cloudy at least 50% of the time.

In [16]:
good_cities = cities_df[cities_df['Temp'] >= 15]
good_cities2 = good_cities[good_cities['Temp'] <= 22]
better_cities = good_cities2[good_cities2['Wind Speed'] >= 5]
best_cities = better_cities[better_cities['Cloudiness'] >= 50]
best_cities = best_cities.dropna()

best_cities

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed
34,tehachapi,35.13,-118.45,16.11,93,90,7.70
51,ahipara,-35.17,173.16,18.33,100,100,10.07
77,castro,-42.48,-73.76,15.00,58,75,5.70
82,marsh harbour,26.54,-77.06,19.14,70,68,10.33
124,ancud,-41.87,-73.83,15.00,58,75,5.70
297,kaeo,-35.10,173.78,20.00,100,100,7.32
312,abu dhabi,24.47,54.37,22.00,78,93,5.10
329,goundam,16.42,-3.66,17.68,17,50,5.04
509,cowra,-33.84,148.69,18.97,63,96,5.79


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = best_cities

hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
34,tehachapi,35.13,-118.45,16.11,93,90,7.70,
51,ahipara,-35.17,173.16,18.33,100,100,10.07,
77,castro,-42.48,-73.76,15.00,58,75,5.70,
82,marsh harbour,26.54,-77.06,19.14,70,68,10.33,
124,ancud,-41.87,-73.83,15.00,58,75,5.70,
297,kaeo,-35.10,173.78,20.00,100,100,7.32,
312,abu dhabi,24.47,54.37,22.00,78,93,5.10,
329,goundam,16.42,-3.66,17.68,17,50,5.04,
509,cowra,-33.84,148.69,18.97,63,96,5.79,


In [30]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Params
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in best_cities.iterrows():
    
    # add city name
    city_name = row["City"]
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    


Retrieving Results for Index 34: tehachapi.
Retrieving Results for Index 51: ahipara.
Retrieving Results for Index 77: castro.
Retrieving Results for Index 82: marsh harbour.
Retrieving Results for Index 124: ancud.
Retrieving Results for Index 297: kaeo.
Retrieving Results for Index 312: abu dhabi.
Retrieving Results for Index 329: goundam.
Retrieving Results for Index 509: cowra.


In [31]:
hotel_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
34,tehachapi,35.13,-118.45,16.11,93,90,7.70,Chateau Builders
51,ahipara,-35.17,173.16,18.33,100,100,10.07,Ahipara WWTP
77,castro,-42.48,-73.76,15.00,58,75,5.70,Borquez Andrade Amado Ernesto
82,marsh harbour,26.54,-77.06,19.14,70,68,10.33,Cable Bahamas (Customer Service Office)
124,ancud,-41.87,-73.83,15.00,58,75,5.70,Cabaña Costa Brava
297,kaeo,-35.10,173.78,20.00,100,100,7.32,Spice Grill Indian Cuisine
312,abu dhabi,24.47,54.37,22.00,78,93,5.10,Eburn
329,goundam,16.42,-3.66,17.68,17,50,5.04,Terrain De Basketball
509,cowra,-33.84,148.69,18.97,63,96,5.79,Parkes St At Brisbane St


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Get coordinates from df

coordinates = [
    (35.13, -118.45),
    (-35.17, 173.16),
    (-42.48, -73.76),
    (26.54, -77.06),
    (-41.87, -73.83),
    (-35.10, 173.78),
    (24.47, 54.37),
    (16.42, -3.66),
    (-33.84, 148.69)
]


In [36]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))